In [3]:
import pandas as pd

df = pd.read_csv('all_lettters_with_entities_final.tsv',sep='\t')

In [4]:
df['filename'] = df['images.reference'].map(lambda s: s.split('\\')[-1])

In [17]:
df['people_old'] = df['people']

df['people']=df['people_split']
df

,part_of.title,images.reference,filename,is_missing,path_to_image,page_type,path_to_text,path_to_handwritten_info,people,handwritten_info,people_split,people_old
0,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\757\0001246757.jpg,0001246757.jpg,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'Spier', 'Haanstra', 'Haan'}",NaN,"{'Spier', 'Haanstra', 'Haan'}","{'Spier', 'Haanstra', 'Haan'}"
1,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\763\0001246763.jpg,0001246763.jpg,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'I.Valkema', '.l. &', 'Bert Haanstra', 'Valke...",NaN,"{'I.Valkema', '.l. &', 'Bert Haanstra', 'Valke...","{'Bert Haanstra', 'E.Spier', 'Valkema', 'I.Val..."
2,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\765\0001246765.jpg,0001246765.jpg,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'| Brusse', 'Brusse', 'P. A. Stoon', 'M. Th. ...",NaN,"{'| Brusse', 'Brusse', 'P. A. Stoon', 'M. Th. ...","{'P. A. Stoon', 'W l', 'Hint Cre', 'M. Th. Bru..."
3,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\767\0001246767.jpg,0001246767.jpg,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,set(),NaN,set(),set()
4,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\769\0001246769.jpg,0001246769.jpg,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'Wim Crouwel', 'T.Theunissen van Maanen', 'Kh...",NaN,"{'Wim Crouwel', 'T.Theunissen van Maanen', 'Kh...","{'Goed Wone', 'Wim Crouwel', 'Kho Liang Ile', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5828,Letters to 'Kristalunie Maastricht' and the 'G...,..\images\jpg\812\0001457812.jpg,0001457812.jpg,False,/ivi/ilps/personal/vprovat/missing_folders_Art...,likely handwritten,/ivi/ilps/personal/vprovat/Valkema_HTR_results...,/ivi/ilps/personal/vprovat/backup_ocr/handwrit...,set(),Page number: 0\nOrientation in degrees: 0\nRot...,set(),set()
5829,Letters to 'Kristalunie Maastricht' and the 'G...,..\images\jpg\813\0001457813.jpg,0001457813.jpg,False,/ivi/ilps/personal/vprovat/missing_folders_Art...,likely handwritten,/ivi/ilps/personal/vprovat/Valkema_HTR_results...,/ivi/ilps/personal/vprovat/backup_ocr/handwrit...,set(),Page number: 0\nOrientation in degrees: 180\nR...,set(),set()
5830,Letters to 'Kristalunie Maastricht' and the 'G...,..\images\jpg\814\0001457814.jpg,0001457814.jpg,False,/ivi/ilps/personal/vprovat/missing_folders_Art...,likely handwritten,/ivi/ilps/personal/vprovat/Valkema_HTR_results...,/ivi/ilps/personal/vprovat/backup_ocr/handwrit...,{'G.w. Reichel'},Page number: 0\nOrientation in degrees: 180\nR...,{'G.w. Reichel'},"{'G.w. Reichel', 'pro'}"
5831,Letters to 'Kristalunie Maastricht' and the 'G...,..\images\jpg\816\0001457816.jpg,0001457816.jpg,False,/ivi/ilps/personal/vprovat/missing_folders_Art...,likely handwritten,/ivi/ilps/personal/vprovat/Valkema_HTR_results...,/ivi/ilps/personal/vprovat/backup_ocr/new_resu...,set(),None,set(),set()


In [6]:
df['people'] = df['people'].fillna('set()')

In [7]:
big = 10**7
def get_next_name(filename):
    number = int(filename.split('.')[0]) + 1 # next one
    if number >= big:
        return '00'+str(number)+'.jpg'
    return '000'+str(number)+'.jpg'

In [8]:
def get_next_file(filename):
    name = get_next_name(filename)
    return df[df['filename']==name]

In [9]:
import json
aka = json.load(open('also_known_as_final.json','r'))

alias = {}
for main_name, names in aka.items():
    for name in names:
        alias[name] = main_name

In [10]:
alias['I.Valkema']

'Sybren Valkema'

In [11]:
def resolve_alias(p):
    cnt = 0
    while p in alias and cnt < 10:
        p = alias[p]
        cnt += 1
    if cnt == 10:
        print('Got a loop in aliases! Starting point: ',p)
    return p

In [12]:
def add_edge(p1, p2, doc_id):
    if p1 == p2:
        return
    
    if (p1, p2) in graph:
        graph[(p1,p2)].append(doc_id)
    else:
        graph[(p1,p2)] = [doc_id]
        
        
def add_edges(people, doc_id):
    '''
    people: list of people found in a document
    doc_id: path to text or another unique identifier
    '''
    if len(people) < 2:
        return
    
    for i, p1_raw in enumerate(people):
        for p2_raw in people[i:]:
            p1 = resolve_alias(p1_raw)
            p2 = resolve_alias(p2_raw)
            add_edge(p1, p2, doc_id)
            add_edge(p2, p1, doc_id)

In [13]:
def extract_id(path):
    return path.split('/')[-1].split('.')[0]

In [14]:
from tqdm.auto import tqdm

In [15]:
def stops_here(cur_df):
    if cur_df.empty:
        return True
    try:
        text = open(cur_df.iloc[0]['path_to_text'],'r').read().lower()
    except:
        print('Failed to find text for: ',cur_df.iloc[0]['path_to_text'])
        text = ''
    if text.find('geachte') != -1:
        return True
    if text.find('geehrter') != -1:
        return True
    if text.find('dear') != -1:
        return True
    return False

In [16]:
graph = {} #{('a','b'):[]}

In [18]:
processed_filenames = set()
for i in tqdm(range(0, len(df))):
    row = df.iloc[i]
    if row['filename'] in processed_filenames: # it was someone else's second page already
        continue 
    processed_filenames.add(row['filename'])

    cur_people = set(eval(row['people'])) if isinstance(row['people'],str) else row['people']

    cur_id = extract_id(row['path_to_image'])
    
    df_next = get_next_file(row['filename'])
    while not stops_here(df_next):
        next_row = df_next.iloc[0]
        processed_filenames.add(next_row['filename'])

        next_people = set(eval(next_row['people'])) if isinstance(next_row['people'],str) else next_row['people']
        cur_people = cur_people.union(next_people)

        cur_id += '-'+ extract_id(next_row['path_to_image'])
        
        df_next = get_next_file(next_row['filename'])
        
    add_edges(list(cur_people), cur_id)

  0%|          | 0/5833 [00:00<?, ?it/s]

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246760.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246761.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246762.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246764.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246766.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246768.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246770.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246772.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246774.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246776.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001246778.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247037.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247041.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247043.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247047.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247049.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247051.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247055.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247057.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247059.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247062.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247063.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247363.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247364.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247366.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247368.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247372.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247378.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247380.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247382.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247384.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247392.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247402.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247605.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247606.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247614.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247616.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247617.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247618.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247620.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247622.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247624.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247631.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001247633.txt
Failed to find text for:  /ivi/i

...

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256140.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256142.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256144.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256146.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256148.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256150.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256152.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256154.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256156.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256158.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001256160.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269349.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269351.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269355.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269360.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269369.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269371.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269373.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269375.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269376.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269377.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001269379.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283599.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283602.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283604.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283608.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283614.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283616.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283628.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001283635.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001284038.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001288153.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001288207.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310205.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310207.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310209.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310211.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310215.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310217.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310219.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310221.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310223.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310229.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001310231.txt
Failed to find text for:  /ivi/i

Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001453786.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001453790.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001453791.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001453792.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001453796.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455764.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455772.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455778.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455780.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455782.txt
Failed to find text for:  /ivi/ilps/personal/vprovat/Valkema_HTR_results/0001455784.txt
Failed to find text for:  /ivi/i

In [19]:
graph

{('Haanstra',
  'Spier'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Spier',
  'Haanstra'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Haanstra',
  'Haan'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Haan',
  'Haanstra'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Spier',
  'Haan'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Haan',
  'Spier'): ['0001246757-0001246758-0001246759-0001246760-0001246761-0001246762'],
 ('Corina Teunisse van Manen',
  'L.Licht'): ['0001246763-0001246764-0001246765-0001246766-0001246767-0001246768-0001246769-0001246770-0001246771-0001246772-0001246773-0001246774-0001246775-0001246776-0001246777-0001246778-0001246779-0001246780', '0001246763-0001246764-0001246765-0001246766-0001246767-0001246768-0001246769-0001246770-0001246771-0001246772-0001246773-0001246774-0001246775-0001246776-0001246777-0001246778-0001

In [20]:
# saving to a readable format

records = []
seen_connections = set()
for people, docs in graph.items():
    person1 = min(people)
    person2 = max(people)
    if (person1, person2) in seen_connections:
        continue
    seen_connections.add((person1,person2))
    records.append({'person1':person1,
                   'person2':person2,
                   'document_ids':docs})
    
df_graph = pd.DataFrame.from_records(records)
df_graph

,person1,person2,document_ids
0,Haanstra,Spier,[0001246757-0001246758-0001246759-0001246760-0...
1,Haan,Haanstra,[0001246757-0001246758-0001246759-0001246760-0...
2,Haan,Spier,[0001246757-0001246758-0001246759-0001246760-0...
3,Corina Teunisse van Manen,L.Licht,[0001246763-0001246764-0001246765-0001246766-0...
4,Corina Teunisse van Manen,Teunissen,[0001246763-0001246764-0001246765-0001246766-0...
...,...,...,...
2600021,James Geart,Sin er,[0001457787-0001457788]
2600022,James Geart,James Grant Saus,"[0001457787-0001457788, 0001457787-0001457788]"
2600023,James Grant Saus-Toten,Sin er,[0001457787-0001457788]
2600024,James Grant Saus,James Grant Saus-Toten,"[0001457787-0001457788, 0001457787-0001457788]"


In [25]:
people = set(df_graph['person1'].to_list()+df_graph['person2'].to_list())
len(people)

15358

In [26]:
import pickle
wikidata_candidates_per_name= pickle.load(open('wikidata_candidates_per_name.p','rb'))


# df_graph['wikidata'] = 

In [28]:
wikidata_candidates_per_name['Durk Valkema']

['http://www.wikidata.org/entity/Q29422094']

In [21]:
df_graph[df_graph['person2'].str.endswith('Mallet')] #& df_graph['person1'].str.startswith('A')]

,person1,person2,document_ids
123216,Albert,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123217,Gary Beecham,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123219,Ann W&,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123223,Durk.,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123224,Harvey Littleton,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123226,Durk Valkema,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123231,Albert Museum,J.V.G.Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123232,J.V.G.Mallet,Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123249,Albert,Mallet,[0001248922-0001248923-0001248924-0001248925-0...
123262,Gary Beecham,Mallet,[0001248922-0001248923-0001248924-0001248925-0...


In [42]:
df_graph[df_graph['person2'].str.endswith('Jansen')]

,person1,person2,document_ids
186947,Ir vd Sijs,Nijkhor Jansen,[0001256051-0001256052-0001256053-0001256054-0...
187619,Mare Sybren Valkema Het Ide Van Het Kline Oven...,Nijkhor Jansen,[0001256051-0001256052-0001256053-0001256054-0...
188290,James Conpen,Nijkhor Jansen,[0001256051-0001256052-0001256053-0001256054-0...
188945,Copier,Nijkhor Jansen,[0001256051-0001256052-0001256053-0001256054-0...
189608,Aems,Nijkhor Jansen,[0001256051-0001256052-0001256053-0001256054-0...
...,...,...,...
3227319,Heet las,Pijp Jansen,[0001307938-0001307939-0001307940-0001307941-0...
3227322,B,Pijp Jansen,[0001307938-0001307939-0001307940-0001307941-0...
3227324,Karel Appel,Pijp Jansen,[0001307938-0001307939-0001307940-0001307941-0...
3227326,Erwin Eisch,Pijp Jansen,[0001307938-0001307939-0001307940-0001307941-0...


In [22]:
df_graph.to_csv('/ivi/ilps/personal/vprovat/valkema_graph_directed_final_final.tsv',sep='\t',index=False)

In [16]:
# df[df['filename']=='0001283499.jpg']

In [17]:
from PIL import Image
def show_image(filename):
    df_target = df[df['filename'] == filename]
    img_path_big = df_target.iloc[0]['path_to_image']
    img_path = img_path_big.replace('TIF','JPG').replace('tif','jpg')
    try:
        img = Image.open(img_path)
    except:
        img = Image.open(img_path_big)
    display(img)
    print(img_path_big)

In [18]:
# show_image('0001246779.jpg')

In [25]:
import networkx as nx


In [26]:
import numpy as np

In [27]:
def num_people(doc_id):
    people = eval(df[df['filename']==doc_id+'.jpg'].iloc[0]['people'])
    return len(people)

In [28]:
def get_weight(doc_ids):
    total = sum([num_people(doc_id) for doc_id in doc_ids.split('-')])
    return total # todo check it

In [29]:
def get_weight_for_list(doc_ids_list):
    return sum([1/get_weight(doc_id) for doc_id in doc_ids_list])

In [30]:
G = nx.Graph()
for item in graph.items():
    if len(set(item[1])) < 3: # pruning the long tail
        continue
    if item[0][0] == 'Valkema' or item[0][1] == 'Valkema': # ambiguous node, not useful
        continue 
    # (('M. Th. Brusse', 'Valkema'), ['0001246763'])
    weight = get_weight_for_list(item[1])
    G.add_edge(item[0][0], item[0][1], weight=weight)# np.log(len(set(item[1]))))

KeyboardInterrupt: 

In [31]:
from pyvis.network import Network
def prepare_network(G1):
    net = Network(notebook=True,cdn_resources='remote')
    net.from_nx(G1)
    net.toggle_physics(True)
    return net

In [ ]:
net = prepare_network(G)
net.repulsion()
net.show('graph-with-handwritten.html')

In [32]:
# now, Valkema-only

Valkema = nx.Graph()
for item in graph.items():
    if len(set(item[1])) < 3: # pruning the long tail
        continue
    if item[0][0] == 'Valkema' or item[0][1] == 'Valkema': # ambiguous node, not useful
        continue 
    if item[0][0] != 'Sybren Valkema' and item[0][1] != 'Sybren Valkema': # Valkema only
        continue 
    # (('M. Th. Brusse', 'Valkema'), ['0001246763'])
    weight = get_weight_for_list(item[1])
    Valkema.add_edge(item[0][0], item[0][1], weight=weight)# np.log(len(set(item[1]))))
    
net = prepare_network(Valkema)
net.repulsion()
net.show('Valkema-with-handwritten.html')

Valkema-with-handwritten.html


In [ ]:
# now, Valkema-only

Valkema = nx.Graph()
for item in graph.items():
    if len(set(item[1])) < 3: # pruning the long tail
        continue
    if item[0][0] == 'Valkema' or item[0][1] == 'Valkema': # ambiguous node, not useful
        continue 
    if not item[0][0].endswith('Valkema') and not item[0][1].endswith('Valkema'): # Valkema only
        continue 
    # (('M. Th. Brusse', 'Valkema'), ['0001246763'])
    weight = get_weight_for_list(item[1])
    Valkema.add_edge(item[0][0], item[0][1], weight=weight)# np.log(len(set(item[1]))))
    
net = prepare_network(Valkema)
net.repulsion()
net.show('Valkema.html')

In [ ]:
# todo: ask Carlotta for her list of people, show her ours, discuss the graph
# ask how she came up with places - extracting from letters or knowing where everyone works?

In [ ]:
# automatic network analysis for digital humanities
# RQs from Carlotta
# find centrality, PageRank etc